<h1> Data Cleaning and Feature Engineering </h2>

In [84]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter
import random
import warnings
warnings.filterwarnings('ignore')

#AWS
import boto3
from botocore.exceptions import ClientError
from io import StringIO

In [85]:
def uploadToS3(result, filename):
    try:
        bucket = 'lendingclubdatastore' # already created on S3
        csv_buffer = StringIO()
        result.to_csv(csv_buffer)
        s3_resource = boto3.resource('s3')
        s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True
    
def readfromS3(filename):
    path = 's3://lendingclubdatastore/'
    bucket = path + filename  # already created on S3
    client = boto3.resource('s3')
    df = pd.read_csv(bucket)
    return df

<h1>Remove outliers</h1>

In [83]:
# loans = pd.read_csv(os.getcwd()+'/files/loan.csv')
loans = readfromS3('loan.csv')

loans = loans[loans['loan_status'].isin(['Fully Paid','Default','Charged Off'])]
loans['loan_status'].unique()
loans = loans[loans['int_rate']<=25]
loans = loans[loans['loan_amnt'] <=35000]
print(loans.shape)
print(max(loans['int_rate']))
print(max(loans['loan_amnt']))

(1270094, 145)
24.99
35000


<h1> Managing the loan status and loan term </h1>

In [86]:
for key, value in Counter(loans['loan_status']).items():
    print(key + ' : ' + str(value) + ' rows')

Fully Paid : 1022121 rows
Charged Off : 247948 rows
Default : 25 rows


In [87]:
loans = loans.replace({'loan_status':'Charged Off'},'Default')
loans = loans.replace({'term':' 36 months'},36)
loans = loans.replace({'term':' 60 months'},60)


loans = loans.replace({'emp_length':'5 years'},5)
loans = loans.replace({'emp_length':'10+ years'},12)
loans = loans.replace({'emp_length':'4 years'},4)
loans = loans.replace({'emp_length':'3 years'},3)
loans = loans.replace({'emp_length':'1 year'},1)
loans = loans.replace({'emp_length':'< 1 year'},0.5)
loans = loans.replace({'emp_length':'8 years'},8)
loans = loans.replace({'emp_length':'2 years'},2)
loans = loans.replace({'emp_length':'6 years'},6)
loans = loans.replace({'emp_length':'9 years'},9)
loans = loans.replace({'emp_length':'7 years'},7)

<h1>Prepare a datasets for borrowers and investors</h1>

In [88]:
loans_borrowers = loans[['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
                        'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status','dti','delinq_2yrs',
                        'earliest_cr_line','mths_since_last_record','open_acc','acc_now_delinq','total_acc',
                        'mths_since_last_major_derog','annual_inc_joint', 'dti_joint','acc_now_delinq','tot_cur_bal','open_acc_6m', 'open_act_il', 
                        'open_il_12m', 'open_il_24m']]

loans_investors = loans[['loan_amnt','funded_amnt', 'funded_amnt_inv','term','int_rate','grade','loan_status','out_prncp', 'out_prncp_inv','total_pymnt', 'total_pymnt_inv',
                         'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries', 'collection_recovery_fee','tot_coll_amt']]

<h1> Handling Null Values </h1>

<h2> Borrowers </h2>

In [89]:
loans_borrowers.columns[loans_borrowers.isnull().any()]

Index(['member_id', 'emp_length', 'dti', 'mths_since_last_record',
       'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m'],
      dtype='object')

In [90]:
member_id = []

for i in range(1,len(loans_borrowers.index)+1):
    member_id.append(i)

loans_borrowers.loc[np.isnan(loans_borrowers['member_id']), 'member_id'] = member_id

emp_length = int(loans_borrowers['emp_length'].mean(skipna=True))
loans_borrowers.loc[np.isnan(loans_borrowers['emp_length']), 'emp_length'] = emp_length

dti_mean = loans_borrowers['dti'].mean(skipna=True)
loans_borrowers.loc[np.isnan(loans_borrowers['dti']), 'dti'] = dti_mean

mts_record_mean = loans_borrowers['mths_since_last_record'].mean(skipna=True)
loans_borrowers.loc[np.isnan(loans_borrowers['mths_since_last_record']), 'mths_since_last_record'] = mts_record_mean

derog_mean = loans_borrowers['mths_since_last_major_derog'].mean(skipna=True)
loans_borrowers.loc[np.isnan(loans_borrowers['mths_since_last_major_derog']), 'mths_since_last_major_derog'] = derog_mean

annual_inc_joint = loans_borrowers['annual_inc_joint'].mean(skipna=True)
loans_borrowers.loc[np.isnan(loans_borrowers['annual_inc_joint']), 'annual_inc_joint'] = annual_inc_joint

dti_joint = loans_borrowers['dti_joint'].mean(skipna=True)
loans_borrowers.loc[np.isnan(loans_borrowers['dti_joint']), 'dti_joint'] = dti_joint

tot_cur_bal = loans_borrowers['tot_cur_bal'].mean(skipna=True)
loans_borrowers.loc[np.isnan(loans_borrowers['tot_cur_bal']), 'tot_cur_bal'] = tot_cur_bal


loans_borrowers.loc[np.isnan(loans_borrowers['open_acc_6m']), 'open_acc_6m'] = 0
loans_borrowers.loc[np.isnan(loans_borrowers['open_act_il']), 'open_act_il'] = 0
loans_borrowers.loc[np.isnan(loans_borrowers['open_il_12m']), 'open_il_12m'] = 0
loans_borrowers.loc[np.isnan(loans_borrowers['open_il_24m']), 'open_il_24m'] = 0


print(loans_borrowers.columns[loans_borrowers.isnull().any()])

uploadToS3(loans_borrowers, 'borrowers.csv')

loans_borrowers.head()

Index([], dtype='object')


,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,...,total_acc,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m
100,1,30000,30000,30000.0,36,22.35,1151.16,D,5.0,MORTGAGE,...,19.0,43.651059,150000.00000,23.380000,0.0,472330.0,1.0,3.0,2.0,2.0
170,2,20000,20000,20000.0,36,7.56,622.68,A,12.0,MORTGAGE,...,19.0,43.651059,190000.00000,11.750000,0.0,515779.0,1.0,2.0,0.0,1.0
186,3,4500,4500,4500.0,36,11.31,147.99,B,12.0,RENT,...,25.0,25.000000,113940.16229,18.631395,0.0,29137.0,2.0,1.0,1.0,1.0
269,4,20000,20000,20000.0,60,17.97,507.55,D,4.0,RENT,...,16.0,43.651059,113940.16229,18.631395,0.0,33356.0,2.0,0.0,0.0,0.0
271,5,6600,6600,6325.0,36,11.31,217.05,B,12.0,RENT,...,13.0,43.651059,113940.16229,18.631395,0.0,26836.0,0.0,2.0,0.0,0.0


<h2> Investors </h2>

In [91]:
loans_investors.columns[loans_investors.isnull().any()]

Index(['tot_coll_amt'], dtype='object')

In [92]:
total_coll_amt = loans_investors['tot_coll_amt'].mean(skipna=True)
loans_investors.loc[np.isnan(loans_investors['tot_coll_amt']), 'tot_coll_amt'] = total_coll_amt

print(loans_investors.columns[loans_investors.isnull().any()])

loans_investors.head()

Index([], dtype='object')


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,loan_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,tot_coll_amt
100,30000,30000,30000.0,36,22.35,D,Fully Paid,0.0,0.0,30026.442500,30026.44,30000.0,26.44,0.0,0.0,0.0,0.0
170,20000,20000,20000.0,36,7.56,A,Fully Paid,0.0,0.0,20215.792430,20215.79,20000.0,215.79,0.0,0.0,0.0,0.0
186,4500,4500,4500.0,36,11.31,B,Fully Paid,0.0,0.0,4549.217149,4549.22,4500.0,49.22,0.0,0.0,0.0,306.0
269,20000,20000,20000.0,60,17.97,D,Fully Paid,0.0,0.0,20013.577333,20013.58,20000.0,13.58,0.0,0.0,0.0,0.0
271,6600,6600,6325.0,36,11.31,B,Fully Paid,0.0,0.0,6622.809000,6346.86,6600.0,22.81,0.0,0.0,0.0,0.0


<h3> Feature Engineering for investors </h3>
<h3> Calculated returns for investor in percentages formula == (p-f/f)*(12/term) </h3>

In [93]:
# principal_amt = loans_investors['funded_amnt'] -  loans_investors['total_pymnt']
loans_investors['return'] = (((loans_investors['total_pymnt'] - loans_investors['funded_amnt'])/loans_investors['funded_amnt']) * (12/loans_investors['term']) ) * 100

uploadToS3(loans_investors, 'investors.csv')

loans_investors.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,loan_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,tot_coll_amt,return
100,30000,30000,30000.0,36,22.35,D,Fully Paid,0.0,0.0,30026.442500,30026.44,30000.0,26.44,0.0,0.0,0.0,0.0,0.029381
170,20000,20000,20000.0,36,7.56,A,Fully Paid,0.0,0.0,20215.792430,20215.79,20000.0,215.79,0.0,0.0,0.0,0.0,0.359654
186,4500,4500,4500.0,36,11.31,B,Fully Paid,0.0,0.0,4549.217149,4549.22,4500.0,49.22,0.0,0.0,0.0,306.0,0.364571
269,20000,20000,20000.0,60,17.97,D,Fully Paid,0.0,0.0,20013.577333,20013.58,20000.0,13.58,0.0,0.0,0.0,0.0,0.013577
271,6600,6600,6325.0,36,11.31,B,Fully Paid,0.0,0.0,6622.809000,6346.86,6600.0,22.81,0.0,0.0,0.0,0.0,0.115197
